Below is the code to scrape bitcoin related tweets

In [ ]:
#!pip install snscrape
import pandas as pd
import snscrape.modules.twitter as sntwitter #scrape tweets
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
import re
import requests
from bs4 import BeautifulSoup
import dateutil

In [ ]:
def scrape_tweets(topic, start_date, end_date, daily_limit):
    df_lst = []
    delta = end_date - start_date
    days = delta.days
    
    spam_words = ['chance', 'Follow', 'follow', 'free', 'link', 'LINK', 'bio', \
                  'BIO', 'Update', 'last price', 'Current {} Price'.format(topic),\
                  'ONLY', 'BLOOMBERG', "report", "Bollinger", 'chart', 'unlock',\
                  'Poll', 'live'] #filter spam tweets by banning keywords

    spam_emojis = ' -👋 -🟩 -🟢 -🟥 -🔍 -📊 -⬅️ -➡️ -🤖 -🔴 -✅ -📈 -💹 -🤖 -🔻 -🖕 -📉 -⬇️ -👇🏻 -❔ '
    for i in range(days+1):
        tweets = []
        start_of_day = (start_date+timedelta(i)).strftime("%Y-%m-%d")
        end_of_day =  (start_date+timedelta(i+1)).strftime("%Y-%m-%d")
        print("scraping "+start_of_day)
        query = "({topic})".format(topic=topic) + ' -'+' -'.join(spam_words)+' ' + spam_emojis +\
        "lang:en until:{end_of_day} since:{start_of_day} -filter:links -filter:replies"\
        .format(start_of_day=start_of_day, end_of_day=end_of_day)

        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            if len(tweets) == daily_limit:
                break
            else:
                second_round_spam_words = [':', '₿', 'price now', 'Price:', 'price in', 'change in', 'price at close', \
                                          'price today', 'was $', '|', '?', '%', 'price of bitcoin is', 'average price of', '=', \
                                          "Today's Bitcoin Price $", "price of bitcoin on", "Let's find an event", "⇄",
                                          "The latest bitcoin price is", "via Chain", "now opening", "bitcoin price is now",\
                                          "contact"]
                to_add = True
                for spam_word in second_round_spam_words:
                    if spam_word.lower() in tweet.content.lower():
                        to_add = False
                        break 
                if to_add:
                    tweets.append([tweet.date, tweet.username, tweet.content])
        df_lst.append(pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet']))
    return pd.concat(df_lst)

def clean_tweet(tweet): 
    tweet = re.sub('@', '', tweet)
    tweet = re.sub('#', '', tweet)
    tweet = re.sub('http\S+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\s+', ' ', tweet)
    return tweet

def get_subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def get_polarity(tweet):
    return TextBlob(tweet).sentiment.polarity
def convert_timezone(date):
    return date.tz_convert('US/Eastern')

In [ ]:
start_date = datetime(2022,10,1)
end_date = datetime(2022,10,7)
df = scrape_tweets("ethereum", start_date, end_date, 100).reset_index(drop=True)

In [ ]:
df['Tweet'] = df.Tweet.apply(clean_tweet)
df['subjectivity'] = df.Tweet.apply(get_subjectivity)
df['polarity'] = df.Tweet.apply(get_polarity)

In [ ]:
x = df.loc[~df.Tweet.str.contains('price [for|of] [Bb]itcoin [on|is]') & ~df.Tweet.str.contains('all exchanges is') \
           & ~df.Tweet.str.contains('=') & ~df.Tweet.str.contains("Today's [Bb]itcoin [Pp]rice $") \
           & ~df.Tweet.str.contains("Let's find an event") \
           & ~df.Tweet.str.contains("The latest bitcoin price is") & ~df.Tweet.str.contains("⇄")\
           & ~df.Tweet.str.contains("via Chain") & ~df.Tweet.str.contains("price of .+ on .+ is")\
           & ~df.Tweet.str.contains("[Mm]arket rank is") & ~df.Tweet.str.contains("Now Opening")\
           & ~df.Tweet.str.contains("[Pp]rice is now") & ~df.Tweet.str.contains("裁定取引")\
           & ~df.Tweet.str.contains("[Pp]rice of bitcoin is now") \
           & ~df.Tweet.str.contains("[Cc]ryptocurrency [Nn]ame [Vv]olume [Pp]rice")\
           & ~df.Tweet.str.contains("[Pp]rice of .+ is currently") & ~df.Tweet.str.contains("contact")]
x.reset_index(drop=True, inplace=True)

In [ ]:
x.to_csv('tweets.csv')


Below is the code to scrap eth related tweets

In [ ]:
def scrape_tweets(topic, start_date, end_date, daily_limit):
    df_lst = []
    delta = end_date - start_date
    days = delta.days
    
    spam_words = ['chance', 'Follow', 'follow', 'free', 'link', 'LINK', 'bio', \
                  'BIO', 'Update', 'last price', 'Current {} Price'.format(topic),\
                  'ONLY', 'BLOOMBERG', "report", "Bollinger", 'chart', 'unlock',\
                  'Poll', 'live', 'sale'] #filter spam tweets by banning keywords

    spam_emojis = ' -👋 -🟩 -🟢 -🟥 -🔍 -📊 -⬅️ -➡️ -🤖 -🔴 -✅ -📈 -💹 -🤖 -📉 -⬇️ -👇🏻 -❔ -📌 -🔗 -⛽ '
    #spam_emojis = ''
    for i in range(days+1):
        tweets = []
        start_of_day = (start_date+timedelta(i)).strftime("%Y-%m-%d")
        end_of_day =  (start_date+timedelta(i+1)).strftime("%Y-%m-%d")
        print("scraping "+start_of_day)
        query = "({topic})".format(topic=topic) + ' -'+' -'.join(spam_words)+' ' + spam_emojis +\
        "lang:en until:{end_of_day} since:{start_of_day} -filter:links -filter:replies"\
        .format(start_of_day=start_of_day, end_of_day=end_of_day)

        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            if len(tweets) == daily_limit:
                break
            else:
                second_round_spam_words = ['price is', 'price now', ':', '|', '(€', '$eth', '%', 'what do you think',\
                                          '- $', 'price prediction is', 'hit me up', 'in comments', 'price of',\
                                          '$', '?', 'tell me', 'price -', 'price update', 'dm', 'check out', 'hire me',\
                                          ' = ', 'nftcommunity']
                to_add = True
                for spam_word in second_round_spam_words:
                    if spam_word.lower() in tweet.content.lower():
                        to_add = False
                        break 
                if to_add:
                    tweets.append([tweet.date, tweet.username, tweet.content])
        daily_df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet']).drop_duplicates(subset = "Tweet")
        daily_df['Tweet'] = daily_df.Tweet.apply(clean_tweet)
        daily_df['subjectivity'] = daily_df.Tweet.apply(get_subjectivity)
        daily_df['polarity'] = daily_df.Tweet.apply(get_polarity)
        df_lst.append(daily_df)
    return pd.concat(df_lst)

In [ ]:
start_date = datetime(2012,12,1)
end_date = datetime(2022,10,7)
df = scrape_tweets("ethereum", start_date, end_date, 1000000).reset_index(drop=True)

In [ ]:
x = df.loc[~df.Tweet.str.contains('price [for|of] [Ee]thereum [on|is]') & ~df.Tweet.str.contains('Technical Analysis') \
           & ~df.Tweet.str.contains("[Pp]rice of .+ is .+ in ") & ~df.Tweet.str.contains("[1.|2.|3.|4.|5.|6.|7.|8.|9.|.10.]")\
           & ~df.Tweet.str.contains("(↑|↓)")]
x.reset_index(drop=True, inplace=True)
x.to_csv('ethereum_tweets.csv')

Below is the code to scrape crypto related tweets

In [ ]:
def scrape_tweets(topic, start_date, end_date, daily_limit):
    df_lst = []
    delta = end_date - start_date
    days = delta.days
    
    spam_words = ['chance', 'Follow', 'follow', 'free', 'link', 'LINK', 'bio', \
                  'BIO', 'Update', 'last price', 'Current {} Price'.format(topic),\
                  'ONLY', 'BLOOMBERG', "report", "Bollinger", 'chart', 'unlock',\
                  'Poll', 'live', 'sale'] #filter spam tweets by banning keywords

    spam_emojis = ' -🟩 -🟢 -🟥 -🔍 -📊 -⬅️ -➡️ -🤖 -🔴 -✅ -📈 -💹 -🤖 -📉 -⬇️ -👇🏻 -❔ -📌 -🔗 -▲ '
    #spam_emojis = ''
    for i in range(days+1):
        tweets = []
        start_of_day = (start_date+timedelta(i)).strftime("%Y-%m-%d")
        end_of_day =  (start_date+timedelta(i+1)).strftime("%Y-%m-%d")
        print("scraping "+start_of_day)
        query = "({topic})".format(topic=topic) + ' -'+' -'.join(spam_words)+' ' + spam_emojis +\
        "lang:en until:{end_of_day} since:{start_of_day} -filter:links -filter:replies"\
        .format(start_of_day=start_of_day, end_of_day=end_of_day)

        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            if len(tweets) == daily_limit:
                break
            else:
                second_round_spam_words = [':', '%', '24h', 'price is', '=', '$', 'price today in', 'price at close', '*',\
                                          'financial solutions', 'vEmpire', 'tradeosiann', 'following his tweet']
                to_add = True
                for spam_word in second_round_spam_words:
                    if spam_word.lower() in tweet.content.lower():
                        to_add = False
                        break 
                if to_add:
                    tweets.append([tweet.date, tweet.username, tweet.content])
        daily_df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet']).drop_duplicates(subset = "Tweet")
        daily_df['Tweet'] = daily_df.Tweet.apply(clean_tweet)
        daily_df['subjectivity'] = daily_df.Tweet.apply(get_subjectivity)
        daily_df['polarity'] = daily_df.Tweet.apply(get_polarity)
        df_lst.append(daily_df)
    return pd.concat(df_lst)

In [ ]:
start_date = datetime(2012,12,1)
end_date = datetime(2022,10, 7)
df = scrape_tweets("crypto", start_date, end_date, 1000).reset_index(drop=True)

In [ ]:
x = df.loc[~df.Tweet.str.contains("[Pp]rice of .+ is") & ~df.Tweet.str.contains("[Rr]equest") & \
          ~df.Tweet.str.contains("Some Mobile Terminal Commands that monitors Crypto-Currency marketcap/price") & \
          ~df.Tweet.str.contains("£[0-9]+") & ~df.Tweet.str.contains("follow4follow") & ~df.Tweet.str.contains("followback") &\
          ~df.Tweet.str.contains("Find BTC") & ~df.Tweet.str.contains("NowPlaying") & ~df.Tweet.str.contains("price for you") &\
          ~df.Tweet.str.contains("followers") & ~df.Tweet.str.contains('[Tt]echnical Analysis') &
          ~df.Tweet.str.contains('[Pp]rediction[s] for 20[0-9][0-9]') & ~df.Tweet.str.contains('PULSEWAVE PRICE') &\
          ~df.Tweet.str.contains("checkout") & ~df.Tweet.str.contains('[Dd]aily [Aa]nalysis') &\
          ~df.Tweet.str.contains("[Pp]rice [0-9].*") & ~df.Tweet.str.contains("[Ww]here .* [Pp]rice of .* be on .* 20[0-9][0-9]") &\
          ~df.Tweet.str.contains("[Pp]rice for .+ is") & ~df.Tweet.str.contains("[Ww]atch this space!") &\
          ~df.Tweet.str.contains("winners") & ~df.Tweet.str.contains("Crypcore") & ~df.Tweet.str.contains("Konios")&\
          ~df.Tweet.str.contains("mana crypto") & ~df.Tweet.str.contains("[Ii]ntroducing [Tt]he [Nn]ew") &\
          ~df.Tweet.str.contains("Although the Lightning Network has experienced growth and development") &\
          ~df.Tweet.str.contains("Ever wonder if you are getting the best price") & ~df.Tweet.str.contains("HEX reward") &\
          ~df.Tweet.str.contains("price predictions .* 20[0-9][0-9]") & ~df.Tweet.str.contains("please comment") &\
          ~df.Tweet.str.contains("[Dd][Mm]") & ~df.Tweet.str.contains("[Ee]merald [Cc]rypto") &\
          ~df.Tweet.str.contains("[Tt]ry it out") & ~df.Tweet.str.contains("[Cc]heck it out")]
x.reset_index(drop=True, inplace=True)
x.to_csv('crypto_tweets.csv')

In [ ]:
'a{b}{c}{d}'.format(b='1', c='2', d='3')

'a123'

Below is the code to scrape ny times news

In [ ]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + ny_times_key
    response = requests.get(url).json()
    time.sleep(6)
    return response

def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline

def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    x = pd.DataFrame(data)
    return x[x.headline.str.contains('[Bb]itcoin') | x.headline.str.contains('[Ee]thereum') |\
  x.headline.str.contains('[Cr]ypto') | x.headline.str.contains('[B]blockchain')] 

In [ ]:
ny_times_key='my_api_key'
df_lst = []
end = datetime.date(2022, 9, 30)
start = datetime.date(2022, 9, 1)
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]
print(months_in_range)
for month in months_in_range:
    respone = send_request(month)
    df_lst.append(parse_response(respone))